# Sentiment Boilerplate

This notebook is a boilerplate for displaying and aggregating metrics about sentiment in a dashboard. In this notebook sentiment is defined as the positive or negative classification of some metric. This notebook is considered a boilerplate because it allows you to plug in your own metrics and then immediately use the dashboard without having to write any UI code. The dashboard UI includes a plot of the total sentiment (positive and negative) and a breakdown plot of the individual sentiment metrics. There is also a historical plot which shows how the individual metrics have changed over time. This plot allows you to interact with a timeline to view the the historical sentiment values within a specific window of time.

**Table of Contents**

1. [Getting Started](#Getting-Started)
1. [Extension Point](#Extension-Point)
1. [Boilerplate Code](#Boilerplate-Code)
    1. [Setup](#Setup)
    1. [Utilities](#Utilities)
    1. [get_dashboard_data](#get_dashboard_data)
    1. [get_historical_sentiment_data](#get_historical_sentiment_data)
1. [UI](#UI)
    1. [Styles](#Styles)
    1. [Connecting To The Backend](#Connecting-To-The-Backend)
    1. [Dashboard Header](#Dashboard-Header)
    1. [Overall Sentiment Widget](#Overall-Sentiment-Widget)
    1. [Sentiment Breakdown Widget](#Sentiment-Breakdown-Widget)
    1. [Historical Sentiment Widget](#Historical-Sentiment-Widget)

<div class="alert alert-info" role="alert" style="margin-top: 10px">
<strong>Note:</strong> Requires Declarative Widgets v0.6.0+.
</div>

## Getting Started

To get started, run all of the cells in this notebook and go into dashboard mode via the dashboard button (<button class="fa fa-dashboard
 icon-play btn btn-xs btn-default"></button>) above. Observe the sample metrics in the widgets of the dashboard. You will be able to plugin your own sentiment metrics to adapt this dashboard to your needs.

This boilerplate contains a simple implementation of [`get_sentiment_counts_during`](#Extension-Point) to illustrate the data structure needed for the boilerplate UI. This is the extension point here you can adapt this dashboard to your particular use case. Returning the proper data structure from the `get_sentiment_counts_during` function is the only thing you need to do to plug your own data into this boilerplate dashboard.

In short, here is a checklist of how to use this notebook.

1. Run all the cells in the notebook and view it as a dashboard. 
1. Read the [Extension Point](#Extension-Point) section to understand how you can adapt this notebook to your own use case.
1. Modify `get_sentiment_counts_during` to connect to your own datasources.
1. Rerun the notebook and see your data in the dashboard.

## Extension Point
Below is the main extension point for bringing in your own data sources to the dashboard. The `get_sentiment_counts_during` function accepts two dates as arguments and returns an array of arrays. The two date arguments are [datetime](https://docs.python.org/3/library/datetime.html) objects used for filtering your data within a certain time window. The return value, an array of arrays, is a collection of metrics and their positive and negative sentiment counts. 


For example, in the call center scenraio a proper return value might be:

```
[
    ['Call Wait Time', 10, 20],
    ['Manager Involvement', 25, 5]
]
```

In this example we define positive `Call Wait Time` when a customer waited for less than 10 minutes. Therefore,  negative `Call Wait Time` is when a custimer waited 10 minutes or more for their call to be handled. Positive `Manager Involvment` is defined as not having to invovle a manager during the call, whereas negative `Manager Involvment` involves a manager.

In [ ]:
def get_sentiment_counts_during(start_date, end_date):
    '''Returns sentiment counts (positive and negative) for indiviudal metrics. 
    '''    
    return [
        ['Metric A', random.randint(50,60), random.randint(0,10)],
        ['Metric B', random.randint(20,30), random.randint(40,50)],
        ['Metric C', random.randint(70,80), random.randint(10,20)]
    ]

This constant allows you to change the length of the current sentiment window. This affects the outcome of [`get_dashboard_data`](#get_dashboard_data) which will affect the total and individual sentiment count charts.

In [ ]:
CURRENT_WINDOW_SIZE_IN_DAYS = 7

This constant controls the number of data points returned by [`get_historical_sentiment_data`](#get_historical_sentiment_data). You can increase or decrease this number to respectively get more or less data points.

In [ ]:
HISTORICAL_WINDOW_SLICES = 30

## Boilerplate Code
### The code below needs no modification to make this boilerplate work. Only modify if you want to extend the capabilites of the boilerplate beyond what is provided.

### Setup
These are imports that bring in python modules needed within the notebook.

In [ ]:
from datetime import datetime, timedelta
import dateutil.parser
import random
from declarativewidgets import channel

In [ ]:
import declarativewidgets as widgets

widgets.init()

These imports bring in frontend elements into the notebook html. There are elements from the [declarativewidgets](https://github.com/jupyter-incubator/declarativewidgets) project as well as [Polymer](https://www.polymer-project.org/1.0/). There is also a custom widget, [urth-timeline](./widgets/urth-timeline), created for this notebook.

In [ ]:
%%html
<link rel='import' is='urth-core-import' href='urth_components/urth-viz-chart/urth-viz-chart.html'>
<link rel='import' is='urth-core-import' href='urth_components/paper-slider/paper-slider.html' 
    package='PolymerElements/paper-slider'>
<link rel='import' is='urth-core-import' href='urth_components/paper-button/paper-button.html' 
    package='PolymerElements/paper-button' >
<link rel='import' is='urth-core-import' href='urth_components/paper-progress/paper-progress.html' 
    package='PolymerElements/paper-progress'>
<link rel='import' is='urth-core-import' href='urth_components/paper-toggle-button/paper-toggle-button.html' 
    package='PolymerElements/paper-toggle-button'>
<link rel='import' is='urth-core-import' href='urth_components/urth-timeline/urth-timeline.html'
    package='/home/jovyan/work/sentiment_boilerplate_demo/widgets/urth-timeline'>

### Utilities
This section is meant for utility functions used within the boilerplate. General functions related to parsing, data cleansing, etc. would belong in this area.

In [ ]:
def parse_date(date_string):
    '''Parses an ISO date string and returns a UTC datetime object.
    '''
    if date_string is None:
        return np.nan
    date_val = dateutil.parser.parse(date_string)
    return datetime.utcfromtimestamp(date_val.timestamp())

### `get_dashboard_data`
This function is the main entry point for the current sentiment view in the dashboard UI. The data returned from this function populates the [overall sentiment](#Overall-Sentiment-Widget) and [historical sentiment](#Historical-Sentiment-Widget) widgets.

In [ ]:
def get_dashboard_data():
    '''Returns the individual sentimnet metrics and the total sentiment
    '''
    channel().set('show-progress', True)
    channel().set('progress-end', 2)
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=CURRENT_WINDOW_SIZE_IN_DAYS)
    channel().set('progress-message', 'Collecting individual sentiment counts.')
    channel().set('progress', 1)
    metrics = get_sentiment_counts_during(start_date, end_date)
    individual_sentiment = {
        'rows': metrics,
        'columns': ['Index', 'Positive', 'Negative'],
        'metadata' : {'interval' : CURRENT_WINDOW_SIZE_IN_DAYS}
    }

    total_positive = 0
    total_negative = 0
    channel().set('progress-message', 'Calculating total sentiment counts.')
    channel().set('progress', 2)
    
    for metric in individual_sentiment['rows']:
        total_positive +=  metric[1]
        total_negative +=  metric [2]

    total_sentiment = {
        'rows': [
            ['Total', total_positive, total_negative]
        ],
        'columns': ['Index', 'Positive', 'Negative'],
        'metadata' : {'interval' : CURRENT_WINDOW_SIZE_IN_DAYS}
    }
    
    channel().set('show-progress', False)
    return {
        'individual' : individual_sentiment,
        'total': total_sentiment
    }

### `get_historical_sentiment_data`
This function is the entry point for the [historical sentiment widget](#Historical-Sentiment-Widget).

In [ ]:
def get_historical_sentiment_data(start_date, end_date):
    '''Returns a dictionary representing the individual sentiment components between the start and end dates.
    
    Args:
        start_date: An ISO standard date string representing the start date of the historical window.
        end_date: An ISO standard date string representing the end date of the historical window.
    '''
    channel().set('show-progress', True)
    channel().set('progress-end', HISTORICAL_WINDOW_SLICES)
    start_date = parse_date(start_date)
    end_date = parse_date(end_date)
    rows = []
    curr_time = end_date
    prev_time = end_date
    index = 0
    delta = (end_date - start_date) / HISTORICAL_WINDOW_SLICES
    column_names = []
    while(curr_time > start_date):
        channel().set('progress-message', 'Calculating historical sentiment.')
        channel().set('progress', index)
        prev_time = curr_time
        curr_time = curr_time - delta
        index = index + 1
        metrics = get_sentiment_counts_during(curr_time, prev_time)
        new_row = [curr_time.isoformat()]
        for metric in metrics:
            new_row.append(metric[1])
            new_row.append(metric[2])
        rows.append(new_row)
        if(len(column_names) == 0):
            column_names = list(map(lambda x: x[0],metrics))
        
    interval = delta.days + round(delta.seconds/(60*60*24), 2)
    labeled_columns = []
    for column_name in column_names:
        labeled_columns.append('Positive {}'.format(column_name))
        labeled_columns.append('Negative {}'.format(column_name))
        
    channel().set('show-progress', False)
    return {
        'rows': rows,
        'columns': ['Index'] + labeled_columns, 
        'metadata' : { 'interval' : interval}
    }

# UI

### Styles
A collection of css styles for the widgets displayed below.

In [ ]:
%%html
<style>
paper-progress, paper-slider {
    display: block;
    width: 100%;
    margin: 20px 0;
}
.center {
    text-align:center !important;
}

.fill {
    width:100%;
}
.timeline-container {
    padding-left:60px;    
}
</style>

### Connecting To The Backend
In the cell below there are two html elements of type `urth-core-function`. These elements have an attribute called `ref` which references functions in the Jupyter Kernel.

In [ ]:
%%html
<urth-core-channel id="defaultChannel"></urth-core-channel>
<template is="urth-core-bind">
    <urth-core-function id="get_dashboard_data_function" ref="get_dashboard_data" 
        result='{{sentiment_data}}'
        delay='500'
        auto>
    </urth-core-function> 
    <urth-core-function id="get_historical_sentiment_data_function" ref="get_historical_sentiment_data" 
        arg-start_date='{{window_start}}'
        arg-end_date='{{window_end}}'
        result='{{historical_data}}'
        delay='500'
        auto>
    </urth-core-function> 
</template>

Initialization code to get variables setup on the client side.

In [ ]:
%%html
<script>
    Urth.whenReady(function() {
        var channel = document.getElementById('defaultChannel');
        // Set the start and end for the timeline widget
        // Initialize the window values to some date string
        channel.set('window_end', new Date().toISOString());
        channel.set('window_start', new Date().toISOString());
        // The color palette used in the visualizations
        channel.set('palette', ['#2166ac','#b2182b', '#4393c3', '#d6604d', '#92c5de', '#f4a582', '#d1e5f0', '#fddbc7']) 
    });
</script>

### Dashboard Header
This cell displays the header for the dashboard. This is where progress updates will be displayed. User input widgets can also be placed here.

In [ ]:
%%html
<template id='banner' is="urth-core-bind">
    <h1>Sentiment Boilerplate</h1>
    <paper-item>
        <p>
            This boilerplate dashboard is for displaying sentiment metrics. To learn more about how you can 
            adapt this dashboard for your use case see the <a href='#Getting-Started'>Getting Started</a> section in notebook mode.
        </p>
    </paper-item>
    <template is='dom-if' if='{{show-progress}}'>
        <paper-progress value="{{progress}}" min='0' max='{{progress-end}}'></paper-progress>
        <paper-item>
            <p class='center fill'>{{progress-message}}</p>
        </paper-item>
    </template>
</template>

### Overall Sentiment Widget
This bar chart displays the overall sentiment to give the user an understanding of whether the general sentiment is positive or negative. The data displayed in this widget comes from the [`get_dashboard_data`](#get_dashboard_data).

In [ ]:
%%html
<template is="urth-core-bind">
    <h2 class='center' >Overall Sentiment (Past <span>{{sentiment_data.total.metadata.interval}}</span> Days)</h2>
    <urth-viz-chart type='bar' palette='{{palette}}' 
        datarows='{{sentiment_data.total.rows}}' 
        columns='{{sentiment_data.total.columns}}'>
    </urth-viz-chart>
</template>

### Sentiment Breakdown Widget
This widget displays the sentiment breakdown for the indiviudal metrics. Each metric is displayed with counts for positive and negative sentiment. The data displayed in this widget comes from the [`get_dashboard_data`](#get_dashboard_data).

In [ ]:
%%html
<template is="urth-core-bind">
        <h2 class='center' >Sentiment Breakdown (Past <span>{{sentiment_data.individual.metadata.interval}}</span> Days)</h2>
        <urth-viz-chart type='bar' palette='{{palette}}' 
            datarows='{{sentiment_data.individual.rows}}' 
            columns='{{sentiment_data.individual.columns}}'>
        </urth-viz-chart>
</template>

### Historical Sentiment Widget
This widget displays the sentiment of the individual metrics over time. The number of data points in this graph can be customized by modifying the `HISTORICAL_WINDOW_SLICES` in the [Extension Point](#Extension-Point) section. The data displayed in this visualization comes from the [`get_historical_sentiment_data`](#get_historical_sentiment_data) function.

In [ ]:
%%html
<template id='timeline_bind' is="urth-core-bind">
    <h2 class='center'>Historical Sentiment</h2>
    <h3 class='center'>
        <span>{{start_label}}</span> to <span>{{end_label}}</span> every <span>{{historical_data.metadata.interval}}</span> days
    </h3>
    <urth-viz-chart type='line' palette="{{palette}}"
        datarows='{{historical_data.rows}}' 
        columns='{{historical_data.columns}}'>
        <urth-viz-col index="0" type="date" ></urth-viz-col>
    </urth-viz-chart>
    <div class='timeline-container'>
        <urth-timeline on-window-changed='update'></urth-timeline>
    </div>            
    <script>
        timeline_bind.update = function(e){
            var channel = document.getElementById('defaultChannel');
            channel.set('window_start', e.detail.window_start.toISOString());
            channel.set('window_end', e.detail.window_end.toISOString());
            channel.set('start_label', e.detail.window_start.toLocaleDateString());
            channel.set('end_label', e.detail.window_end.toLocaleDateString());
        }
    </script>
</template>